In [1]:
import random
import os

import numpy as np
import pandas as pd
from torch import optim
import torch

import torch.nn as nn
from torch.nn.modules.container import Sequential

from tqdm import tqdm

import gc

class CFG:
    dataPath = "./open/"
    trainPath = dataPath+'train.csv'
    testPath = dataPath+'test.csv'
    submission = dataPath+'sample_submission.csv'
    outPath = dataPath+'processed/'
    weightsavePath = dataPath+'weights/'
    
    device = 'cuda'
    
def seedEverything(random_seed):
    torch.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed) # if use multi-GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    random.seed(random_seed)
    os.environ['PYTHONHASHSEED'] = str(random_seed)
    np.random.seed(random_seed)    
    # np.random.seed(random_seed)

class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(56, 512)
        self.layer1 = self.make_layers(512, num_repeat=300)
        self.relu = nn.ReLU(inplace=True)
        self.fc5 = nn.Linear(512, 14)


    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.layer1(x)
        x = nn.Dropout(0.2)(x)
        x = self.fc5(x)
        return x

    def make_layers(self, value, num_repeat):
        layers = []
        for _ in range(num_repeat):
            layers.append(nn.Linear(value, value))
            layers.append(nn.ReLU(inplace=True))
        return nn.Sequential(*layers)

def numpy_to_tensor(variable):
    x = variable.values
    x = np.array(x, dtype=np.float32)
    x = torch.from_numpy(x)
    return x

def pandas_to_tensor(variable):
    return torch.tensor(variable.values)

def train_one_epoch(model, train_batch, criterion, optimizer, train_X, train_Y, device):
    model.train()
    train_loss = 0.0
    for i in range(train_batch):
        start = i * batch_size
        end = start + batch_size
        input = train_X[start:end].to(device, dtype=torch.float)
        label = train_Y[start:end].to(device, dtype=torch.float)

        input, label = input.to(device), label.to(device)
        outputs = model(input).squeeze()
        loss = criterion(outputs, label)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    print(f"train_loss : {train_loss}")
    
def val_one_epoch(model, val_batch, criterion, val_X, val_Y, device):
    model.eval()
    with torch.no_grad():
        val_loss = 0.0
        for i in range(val_batch):
            start = i * batch_size
            end = start + batch_size
            input = val_X[start:end].to(device, dtype=torch.float)
            label = val_Y[start:end].to(device, dtype=torch.float)

            input, label = input.to(device), label.to(device)
            outputs = model(input).squeeze()
            loss = criterion(outputs, label)
            val_loss += loss.item()
    print(f"val_loss : {val_loss}")
    
def datapreparation(train_df):
    # shuffle
    valset_ratio = 0.15
    train_df = train_df.sample(frac=1)

    train_df_X = train_df.filter(regex='X')
    train_df_Y = train_df.filter(regex='Y')

    valset_num = round(len(train_df_Y)*valset_ratio)

    val_df_X = pandas_to_tensor(train_df_X.iloc[:valset_num])
    val_df_Y = pandas_to_tensor(train_df_Y.iloc[:valset_num])    
    train_df_X = pandas_to_tensor(train_df_X.iloc[valset_num:])
    train_df_Y = pandas_to_tensor(train_df_Y.iloc[valset_num:])

    return train_df_X, train_df_Y, val_df_X, val_df_Y

if __name__ == '__main__':
    seedEverything(42)
    train_df = pd.read_csv(CFG.trainPath)    
    train_df_X, train_df_Y, val_df_X, val_df_Y = datapreparation(train_df)

    model = NeuralNet()
    print(model)
    model = model.to(CFG.device)
    optimizer = optim.Adam(model.parameters(),lr=3e-4)
    criterion = nn.L1Loss().cuda()

    num_epochs = 300
    batch_size = 2048
    
    train_batch = len(train_df_X) // batch_size
    val_batch = len(val_df_X) // batch_size
    
    for epoch in range(num_epochs):
        print(f"epoch:{epoch}")
        train_one_epoch(model, train_batch, criterion, optimizer, train_df_X, train_df_Y, CFG.device)
        val_one_epoch(model, val_batch, criterion, val_df_X, val_df_Y, CFG.device)
        # gc.collect()
#         torch.save(model.state_dict(), CFG.weightsavePath+f'{epoch}_neuralnet.pt')
        print('\n')

NeuralNet(
  (fc1): Linear(in_features=56, out_features=512, bias=True)
  (layer1): Sequential(
    (0): Linear(in_features=512, out_features=512, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=512, out_features=512, bias=True)
    (5): ReLU(inplace=True)
    (6): Linear(in_features=512, out_features=512, bias=True)
    (7): ReLU(inplace=True)
    (8): Linear(in_features=512, out_features=512, bias=True)
    (9): ReLU(inplace=True)
    (10): Linear(in_features=512, out_features=512, bias=True)
    (11): ReLU(inplace=True)
    (12): Linear(in_features=512, out_features=512, bias=True)
    (13): ReLU(inplace=True)
    (14): Linear(in_features=512, out_features=512, bias=True)
    (15): ReLU(inplace=True)
    (16): Linear(in_features=512, out_features=512, bias=True)
    (17): ReLU(inplace=True)
    (18): Linear(in_features=512, out_features=512, bias=True)
    (19): ReLU(inplace=

epoch:0
train_loss : 279.829402923584
val_loss : 34.23571968078613


epoch:1
train_loss : 170.71406888961792
val_loss : 9.123870372772217


epoch:2
train_loss : 42.128963232040405
val_loss : 3.7899526357650757


epoch:3
train_loss : 28.612062454223633
val_loss : 3.3828247785568237


epoch:4
train_loss : 25.65065908432007
val_loss : 3.0818779468536377


epoch:5
train_loss : 24.488911032676697
val_loss : 2.991361618041992


epoch:6
train_loss : 23.750900626182556
val_loss : 2.912130117416382


epoch:7
train_loss : 23.213498950004578
val_loss : 2.883626341819763


epoch:8
train_loss : 22.76290464401245
val_loss : 2.8145865201950073


epoch:9
train_loss : 22.265307068824768
val_loss : 2.730901837348938


epoch:10
train_loss : 21.72695541381836
val_loss : 2.6828659772872925


epoch:11
train_loss : 21.298901319503784
val_loss : 2.629786729812622


epoch:12
train_loss : 20.9010488986969
val_loss : 2.5688050985336304


epoch:13
train_loss : 20.609336972236633
val_loss : 2.5519174337387085


ep

KeyboardInterrupt: 

In [2]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cuda


In [3]:
class CFG:
    dataPath = "./open/"
    trainPath = dataPath+'train.csv'
    testPath = dataPath+'test.csv'
    submission = dataPath+'sample_submission.csv'
    outPath = './output/'
    weightsavePath = './weights/'
    device = 'cuda'

In [4]:
def seedEverything(random_seed):
    torch.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)
#     torch.cuda.manual_seed_all(random_seed) # if use multi-GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    random.seed(random_seed)
    os.environ['PYTHONHASHSEED'] = str(random_seed)
    np.random.seed(random_seed)    
    np.random.seed(seed)
    random.seed(seed)


In [5]:
from sklearn.model_selection import train_test_split
data_df = pd.read_csv(CFG.trainPath)
train_df, val_df = train_test_split(data_df,test_size=0.1)


In [6]:
from torch.utils.data import Dataset

def numpy_to_tensor(variable):
    x = variable.values
    x = np.array(x, dtype=np.float32)
    x = torch.from_numpy(x)
    return x

class CustomDataset(Dataset):
    def __init__(self, df, is_test=False):
        super().__init__()
        self.df = df
        self.is_test = is_test
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self,idx):
        
        label = self.df.filter(regex='Y')
        input_data = self.df.filter(regex='X')
        if is_test == True:
            return numpy_to_tensor(input_data)
        return numpy_to_tensor(input_data), numpy_to_tensor(label)
        

In [7]:
from torch.utils.data import DataLoader
train_set = CustomDataset(df=train_df)
val_set = CustomDataset(df=val_df)

loader_train = DataLoader(dataset=train_set,batch_size=1024,shuffle=True)
loader_val = DataLoader(dataset=val_set,batch_size=1024,shuffle=False)

In [8]:
class Mymodel(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(56, 512)
        self.layer1 = self.make_layers(512, num_repeat=300)
        self.relu = nn.ReLU(inplace=True)
        self.fc5 = nn.Linear(512, 14)


    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.layer1(x)
        x = nn.Dropout(0.2)(x)
        x = self.fc5(x)
        return x

    def make_layers(self, value, num_repeat):
        layers = []
        for _ in range(num_repeat):
            layers.append(nn.Linear(value, value))
            layers.append(nn.ReLU(inplace=True))
        return nn.Sequential(*layers)



def train_one_epoch(model, train_batch, criterion, optimizer, train_X, train_Y, device):
    model.train()
    train_loss = 0.0
    for i in range(train_batch):
        start = i * batch_size
        end = start + batch_size
        input = train_X[start:end].to(device, dtype=torch.float)
        label = train_Y[start:end].to(device, dtype=torch.float)

        input, label = input.to(device), label.to(device)
        outputs = model(input).squeeze()
        loss = criterion(outputs, label)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    print(f"train_loss : {train_loss}")
    
def val_one_epoch(model, val_batch, criterion, val_X, val_Y, device):
    model.eval()
    with torch.no_grad():
        val_loss = 0.0
        for i in range(val_batch):
            start = i * batch_size
            end = start + batch_size
            input = val_X[start:end].to(device, dtype=torch.float)
            label = val_Y[start:end].to(device, dtype=torch.float)

            input, label = input.to(device), label.to(device)
            outputs = model(input).squeeze()
            loss = criterion(outputs, label)
            val_loss += loss.item()
    print(f"val_loss : {val_loss}")


    

In [9]:
test_df = pd.read_csv(CFG.testPath)
test_df_XX = test_df.filter(regex='X')
# test_df_X = pandas_to_tensor(test_df.iloc[:])
test_df_XX

,X_01,X_02,X_03,X_04,X_05,X_06,X_07,X_08,X_09,X_10,...,X_47,X_48,X_49,X_50,X_51,X_52,X_53,X_54,X_55,X_56
0,68.504,103.321,76.67,1,101.867,73.963,30.51,63.57,239.80,0.0,...,1,1,17227.63,138.130429,129.460682,141.506570,133.427229,129.711498,133.138096,121.859684
1,67.485,103.320,69.37,1,101.992,67.845,28.03,116.99,189.23,0.0,...,1,1,17134.53,136.148839,128.266277,145.911745,131.196417,132.411480,133.629025,124.178623
2,69.524,103.320,68.97,1,101.884,77.022,29.65,205.68,214.93,0.0,...,1,1,14860.83,120.447446,119.988804,132.099908,120.450155,130.051708,128.252972,114.475628
3,69.524,103.320,65.87,1,101.866,73.963,28.15,103.38,180.80,0.0,...,1,1,15252.53,133.994695,125.069180,147.507669,123.142653,125.963665,139.666592,126.589253
4,73.603,103.321,66.67,1,101.891,74.983,29.92,71.20,231.93,0.0,...,1,1,10752.23,137.918202,135.116192,138.600473,127.173033,137.252712,134.411335,124.020016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39603,68.504,103.320,63.97,1,103.157,68.864,29.49,116.35,284.16,0.0,...,1,1,62123.53,127.741246,126.494312,139.119905,125.271109,128.284572,140.176945,128.292843
39604,68.504,103.320,61.37,1,103.137,68.864,32.29,116.28,272.41,0.0,...,1,1,61844.13,127.767377,124.062809,138.238664,119.879393,127.322529,137.312047,131.570614
39605,69.524,103.320,63.67,1,103.149,69.884,30.00,113.05,295.54,0.0,...,1,1,60277.53,128.593640,124.774037,138.659624,123.999571,126.075542,135.656132,127.671108
39606,67.485,103.321,61.77,1,103.148,67.845,32.05,115.05,267.26,0.0,...,1,1,60236.73,121.110646,125.471699,134.989984,120.889578,129.296909,132.673977,131.882893


In [10]:
test_data = pandas_to_tensor(test_df_XX)

NameError: name 'pandas_to_tensor' is not defined

In [ ]:
if __name__ == '__main__':
    seedEverything(42)
    train_df = pd.read_csv(CFG.trainPath)    
    train_df_X, train_df_Y, val_df_X, val_df_Y = datapreparation(train_df)

    model = NeuralNet()
    model = model.to(CFG.device)
    optimizer = optim.Adam(model.parameters(),lr=3e-4)
    criterion = nn.L1Loss().cuda()

    num_epochs = 300
    batch_size = 2048
    
    train_batch = len(train_df_X) // batch_size
    val_batch = len(val_df_X) // batch_size
    
    for epoch in range(num_epochs):
        print(f"epoch:{epoch}")
        train_one_epoch(model, train_batch, criterion, optimizer, train_df_X, train_df_Y, CFG.device)
        val_one_epoch(model, val_batch, criterion, val_df_X, val_df_Y, CFG.device)
        # gc.collect()
        torch.save(model.state_dict(), CFG.weightsavePath+f'{epoch}_neuralnet.pt')
        print('\n')
#https://wooono.tistory.com/102

In [ ]:
# def predict(model, test_loader, device):
#     model.to(device)
#     model.eval()
#     model_preds = []
#     with torch.no_grad():
#         for data in tqdm(iter(test_loader)):
#             data = data.float().to(device)
            
#             batch_pred = model(data)
            
#             model_preds += batch_pred.argmax(1).detach().cpu().numpy().tolist()
    
#     return model_preds

test_batch = len(test_df_XX) // batch_size
model.eval()
preds=[]
with torch.no_grad():
    val_loss = 0.0
    for i in range(test_batch+1):
        start = i * batch_size
        end = start + batch_size
        print(end)
        if end>len(test_data):
#             print("check")
            end = len(test_data)
            
        input = test_data[start:end].to(device, dtype=torch.float)
#         label = val_Y[start:end].to(device, dtype=torch.float)

        input= input.to(device)
        outputs = model(input).squeeze()
#         print(outputs)
        preds += outputs.detach().cpu().numpy().tolist()
#         preds.append(outputs
#         loss = criterion(outputs, label)
#         val_loss += loss.item()
# print(f"val_loss : {val_loss}")

In [ ]:
test_batch*batch_size

In [ ]:
len(test_data)

In [ ]:
# preds
preds_np=np.array(preds)
preds_np.shape

# preds_np = preds.detach().cpu().numpy()

In [ ]:
preds_np

In [ ]:
submission_df = pd.read_csv(CFG.submission)


In [ ]:
submission_df.iloc[:,1:]

In [ ]:
submission_df.to_csv('./submit.csv', index=False)

In [ ]:
def load_all_models(n_models):
  all_models=list()
  for i in range(n_models):
    filename='/content/drive/MyDrive/lg aimers/models/'+str(i+1)+'.h5'
    model=load_model(filename)
    all_models.append(model)

    print('>loaded %s' %filename)
  return all_models
  
def define_stacked_model(members):
  # update all layers in all models to not be trainiable
  for i in range(len(members)):
    model=members[i]
    for layer in model.layers:
      layer.trainable=False
      # rename to avoid 'unique layer name' issue
      layer._name = 'ensemble_' + str(i+1) + '_' + layer.name
  
  # define multi-handed input
  ensemble_visible=[model.input for model in members]
  ensemble_outputs=[model.output for model in members]

  y=Average()(ensemble_outputs)

  model=Model(inputs=ensemble_visible,outputs=y,name='ensemble')

  keras.utils.plot_model(model,show_shapes=True,to_file='/content/drive/MyDrive/lg aimers/model_graph.jpg')

  model.compile(loss='mae',optimizer='adam')
  return model

def fit_stacked_model(model,trainX,valX,trainY,valY):
  x_train=[trainX for _ in range(len(model.input))]
  x_val=[valX for _ in range(len(model.input))]
  y_train=trainY
  y_val=valY

  es=keras.callbacks.EarlyStopping(monitor='val_loss',patience=20)
  checkpoint_filepath='/content/drive/MyDrive/lg aimers/ensemble_model.h5'
  cp=keras.callbacks.ModelCheckpoint(
      filepath=checkpoint_filepath,
      monitor='val_loss',
      save_best_only=True
  )
  model.fit(x_train,y_train,epochs=1,validation_data=(x_val,y_val),callbacks=[es,cp],batch_size=32)

def evaluate_stacked_model(model,inputX,y_val):
  x_val=[inputX for _ in range(len(model.input))]
  return model.evaluate(x_val,y_val)

def predict_stacked_model(model,inputX):
  x_test=[inputX for _ in range(len(model.input))]
  return model.predict(x_test)

In [ ]:
n_members=5
members=load_all_models(n_members)
print('Loaded %d members' %len(members))

stacked_model=define_stacked_model(members)
fit_stacked_model(stacked_model,x_train,x_val,y_train,y_val)

In [ ]:
evaluate_stacked_model(stacked_model,x_val,y_val)


In [ ]:
result=predict_stacked_model(stacked_model,test)


In [ ]:
submission_pd=pd.read_csv('/content/drive/MyDrive/lg aimers/sample_submission.csv')


In [ ]:
for idx,col in enumerate(submission_pd.columns):
  if col=='ID':
    continue
  submission_pd[col]=result[:,idx-1]

In [ ]:
submission_pd.to_csv('/content/drive/MyDrive/lg aimers/submission.csv',index=False)